In [1]:
import scipy
import scipy.io
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
import os
from feature_extraction import features_estimation
import pandas as pd

## Plot function
#### Plots 2 different signals
#### Used to show the difference between the base and resampled signals

This is a helper function that can be used to plot our signals and check that a modification has been applied.
It takes in the original signal and the modified one and plots both on the same axis

In [10]:
# Define the function to plot a graph
def plot_signal_modification(time, original_signal, time1, new_signal):
    """
    Function to plot 2 simple graph.

    Parameters:
    - time:             Duration of the signal in seconds
    - original_signal:  Array containing the readings for the original signal
    - new_signal:       Array containing the readings for the new signal
    - time1:            Duration of the signal in seconds
    """

    # Plotting the original EMG signal
    plt.figure(figsize=(10, 5))
    plt.plot(time, original_signal, color='b', linewidth=2)
    plt.plot(time1, new_signal, color='r', linewidth=1)

    # Adding labels and title
    plt.xlabel("Time (s)")
    plt.ylabel("EMG Signal (mV)")
    plt.title("EMG Signal Over Time")
    plt.grid(True)


    # time_resampled = np.linspace(0, time[-1], target_length)
    
    plt.show()


## Resizing function
#### Used to modify the length of the signal for one subject 
This function can be used to resize the the EMG signals for a specific subject.

In [11]:
def uniformize_data_subject(file_path, sub_num, target_length):
    

    num_sensors = 24
    # target_length = 1500
    sampling_rate = 5120

    # Load the MATLAB file
    mat_data = scipy.io.loadmat(file_path)
    data = mat_data['processed_data']
    set_of_all_gesture_by_subject = np.array(data)

    gestures = set_of_all_gesture_by_subject.shape[1]
    repetitions = set_of_all_gesture_by_subject.shape[0]


    # we access all repetitions of the 7 types of hand gestures
    for i in range(repetitions): # shape (28,)

        # we access a specific gesture type
        for j in range(gestures):

            # hand_gesture_type contains the electrovolts reading of the 24 sensors placed on the subjects arms at the ith repetition of the jth hand gesture type
            hand_gesture_type = set_of_all_gesture_by_subject[i,j]    # shape(....,24)
            
            # we check if the sampling length of the signals of this gesture matches the desired length
            if hand_gesture_type.shape[0] != target_length:

                # if it isn't we create a new array of the appropriate size
                new_gesture = np.zeros((target_length, num_sensors))

                # for all sensors we modify the expand the length of the gesture recorded
                for k in range(num_sensors):
                    sample_to_alter = hand_gesture_type[:,k]  # shape (..., 1)
                    resampled_signal = signal.resample(sample_to_alter , target_length)
                    new_gesture[:,k] = resampled_signal

                    if i == 1 and j == 1 and k == 1:
                        time = np.arange(len(sample_to_alter)) / sampling_rate
                        time2 = np.arange(len(resampled_signal)) / sampling_rate
                        plot_signal_modification(time, sample_to_alter, time2, resampled_signal)


                    print( f"converted subject {sub_num} repetition {repetitions} gestures {j} channel {k} ")
                    
                set_of_all_gesture_by_subject[i,j] = new_gesture

   
    output_data = {'processed_data': set_of_all_gesture_by_subject}
    output_dir = "/Users/chrisdollo/Documents/coding_projects/EMG/Data/resized for 1500"
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    
    output_path = os.path.join(output_dir, 'processed_gestures_subject'+ str(sub_num)+ '.mat')

    # Save the file
    savemat(output_path, output_data)

## Dataset Creator

#### This function is used to create the dataset that will be used for modeling

In [ ]:
def create_dataset(num_of_subject = 0, repetitions = 10, gestures = 7, sensors = 24):
    """
    This function creates a matfile that contains hand gesture data from a given range of subjects
    the data is formated as (x, 1500, 24) where x corresponds to the mnumber of subjects times 28
    """

    subject_row_map = {
        3: (0, 28),
        4: (28, 56),
        5: (56, 84),
        6: (0, 28),
        7: (28, 56),
        8: (56, 84),
        9: (0, 28),
        10: (28, 56),
        11: (56, 84),
        12: (0, 28),
        13: (28, 56),
        14: (56, 84),
        15: (0, 28),
        16: (28, 56),
        17: (56, 84),
    }


    # load the matlab table
    table_path = f"/Users/chrisdollo/Documents/coding_projects/EMG/Matlab/gestureTable_clean_deep_learning.mat"
    table = scipy.io.loadmat(table_path)
    cell_array = np.array(table['finalCellArray'])  # shape (84, 7) and each cell is (1500, 24)

    for subject in range(1, num_of_subject):

        
        
        if subject not in [0,1,2,21,28,32,37,40,41,44,52]:
            print(subject)
            try:

                # we load the hand gesture data from individual subject 
                file_path = f"/Users/chrisdollo/Documents/coding_projects/EMG/Data/resized for 1500/processed_gestures_subject{subject}.mat"
                mat_file = scipy.io.loadmat(file_path)
                cell_array_gesture = mat_file['processed_data']
                cell_array_gesture = np.array(cell_array_gesture)           # contains the (28, 7)

                

                # we write the subject data to the appropriate rows and columns in our final table
                cell_array[(subject_row_map[subject][0]):(subject_row_map[subject][1]), : ] = cell_array_gesture

            # we uniformize the number 
            except FileNotFoundError:
                # uniformize_data_subject(f"/Users/chrisdollo/Documents/coding_projects/EMG/Data/processed_gestures_subject{subject}.mat", subject, target_length=1500) 
                print(Error)
                # subject-=1


    # we save the new data to our final file 
    output_data = {'processed_data': cell_array}
    output_dir = "/Users/chrisdollo/Documents/coding_projects/EMG/Data/resized for 1500"
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
        
    output_path = os.path.join(output_dir, 'final_data.mat')
    savemat(output_path, output_data)



In [5]:
def extract_features(file_path, table_path, subject_row_map, subject, sampling_rate, window_size, step_size):

    # load the matlab table
    table = scipy.io.loadmat(table_path)
    cell_array = table['finalCellArray']  # shape (84, 7)

    # Load the MATLAB file
    mat_data = scipy.io.loadmat(file_path)
    data = mat_data["processed_data"]
    set_of_all_gesture_by_subject = np.array(data)
    
    gestures = set_of_all_gesture_by_subject.shape[1]
    repetitions = set_of_all_gesture_by_subject.shape[0]
    sensors = (set_of_all_gesture_by_subject[0][0].shape)[1]

    print("Number of gestures detected is: ", gestures, "\n number of repetitions is detected is: ", repetitions, " \nnumber of sensors detected is: ", sensors)

    # Get the row range to write to
    row_start, row_end = subject_row_map[subject]
    row_index = row_start

    ## acees the whole table 
    print(cell_array.shape)

    # acccess
    print(cell_array[0].shape)

    # access a hand gesture
    print(cell_array[0][0].shape)

    # sensors are rows, features are columns

    # access all features of a sensor
    print(cell_array[0][0][0,:].shape)


    print(row_start)

    # all gestures types
    for gesture_idx in range(gestures): # repeats 7 times
        for repetition_idx in range(repetitions): # repeats 28 times

            feature_cell = [[None for _ in range(18)] for _ in range(sensors)]

            for sensor_idx in range(sensors): # repeats 24 times

                # extract the 18 features 
                emg_features, features_names = features_estimation(set_of_all_gesture_by_subject[repetition_idx][gesture_idx][:,sensor_idx],  
                                                                   "Gesture " + str(gesture_idx+1) + " repetition " + str(repetition_idx+1) + 
                                                                   " channel " + str(sensor_idx+1)  , sampling_rate, window_size, 
                                                                   step_size)

                # Flatten and convert to list
                emg_features = emg_features.to_numpy().flatten().tolist()
                feature_cell[sensor_idx] = emg_features

            # Store the 24 by 28 gesture
            cell_array[row_index, gesture_idx] = np.array(feature_cell, dtype=object)
            row_index += 1

            print("Wrote for gesture: ", gesture_idx," repetition ", repetition_idx, " this was at row ", row_index)

        row_index = row_start
            

                
                
    # Save back to .mat file
    scipy.io.savemat(table_path, {'finalCellArray': cell_array})
    print(f"✅ Saved updated feature data for subject {subject}")




In [13]:
# num_of_subject = 6

# subject_row_map = {
#     3: (0, 27),
#     4: (28, 55),
#     5: (56, 83),
#     # ... define ranges for all subjects
# }

# for subject in range(1,num_of_subject):
#     if subject not in [0,1,2,21,28,32,37,40,41,44,52]:
#         # file_path = "E:\Chris\putEMG Project\Data\\2_uniformized_data\processed_gestures_subject"+ str(subject)+ ".mat"

#         file_path = "/Users/chrisdollo/Downloads/processed_gestures_subject"+ str(subject)+ ".mat"
#         table_path = "/Users/chrisdollo/Documents/coding_projects/EMG/Matlab/gestureTable_clean.mat"
        
#         extract_features(file_path, table_path, subject_row_map, subject, 5120, 15375, 15376)




# file_path = "E:\Chris\putEMG Project\Data\\2_uniformized_data\processed_gestures_subject"+ str(num_of_subject-1)+ ".mat"
# table_path = "E:\Chris\putEMG Project\Code\Matlab\AllFeaturesTableMaker.m"

# file_path = "/Users/chrisdollo/Downloads/processed_gestures_subject3.mat"
# table_path = "/Users/chrisdollo/Documents/coding_projects/EMG/Matlab/gestureTable_clean.mat"
    



# print(file_path)
# print(table_path)


## Call functions Here

In [ ]:
# resize subject 3 data to 1500
# uniformize_data_subject("/Users/chrisdollo/Documents/coding_projects/EMG/Data/processed_gestures_subject3.mat", sub_num=3, target_length=1500)